<h1>Importing Libraries</h1>

In [1]:

import pandas as pd #used for data manipulation
import numpy as np #used for numerical analysis
from collections import Counter as c #returns counts of classes
import matplotlib.pyplot as plt #used for data visualization
import seaborn as sns #used for data visualization
from sklearn.metrics import accuracy_score, confusion_matrix #model performance
from sklearn.model_selection import train_test_split #Splits data in random train and test array
from sklearn.preprocessing import LabelEncoder #encoding the levels of categorical featues
from sklearn.linear_model import LogisticRegression #Classification ML  Algorithm
import pickle #python object hirearchy is converted into a byte stream


<h1>Loading Dataset</h1>

In [2]:
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pMol2rEOsCdL7QLqxuK9GJgbU9mW_nYnOFFnRSUbBBLm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'earlydetectionofchronickidneydise-donotdelete-pr-yy3wixhxgsf20s'
object_key = 'chronickidneydisease.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [3]:
#Drop is used for dropping the column
data.drop(['id'],axis=1,inplace=True)

<h2>Renaming the columns</h2>

In [4]:
#return all the column names
data.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [5]:
#manually giving the names of the columns
data.columns = ['age','blood_pressure','specific_gravity','albumin','sugar','red_blood_cells','pus_cell',
                'pus_cell_clumps','bacteria','blood glucose random','blood_urea','serum_creatinine','sodium','potassium','hemoglobin','packed_cell_volume','white_blood_cell_count','red_blood_cell_count','hypertension','diabetesmellitus','coronary_artery_disease','appetite','pedal_edema','anemia','class']
data.columns

Index(['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar',
       'red_blood_cells', 'pus_cell', 'pus_cell_clumps', 'bacteria',
       'blood glucose random', 'blood_urea', 'serum_creatinine', 'sodium',
       'potassium', 'hemoglobin', 'packed_cell_volume',
       'white_blood_cell_count', 'red_blood_cell_count', 'hypertension',
       'diabetesmellitus', 'coronary_artery_disease', 'appetite',
       'pedal_edema', 'anemia', 'class'],
      dtype='object')

In [6]:
#info will give the summary of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      391 non-null    float64
 1   blood_pressure           388 non-null    float64
 2   specific_gravity         353 non-null    float64
 3   albumin                  354 non-null    float64
 4   sugar                    351 non-null    float64
 5   red_blood_cells          248 non-null    object 
 6   pus_cell                 335 non-null    object 
 7   pus_cell_clumps          396 non-null    object 
 8   bacteria                 396 non-null    object 
 9   blood glucose random     356 non-null    float64
 10  blood_urea               381 non-null    float64
 11  serum_creatinine         383 non-null    float64
 12  sodium                   313 non-null    float64
 13  potassium                312 non-null    float64
 14  hemoglobin               3

<h1>Traget Column</h1>

In [7]:
#find the unique elements of the array
data['class'].unique()


array(['ckd', 'ckd\t', 'notckd'], dtype=object)

<h1>Rectifying the target column</h1>

In [8]:
#replace is used of renaming
data['class'] = data['class'].replace("ckd\t","ckd")
data['class'].unique()

array(['ckd', 'notckd'], dtype=object)

In [9]:
#only fetch the object  type columns
catcols = set(data.dtypes[data.dtypes=='O'].index.values)
print(catcols)

{'red_blood_cells', 'pedal_edema', 'diabetesmellitus', 'appetite', 'hypertension', 'anemia', 'bacteria', 'pus_cell_clumps', 'red_blood_cell_count', 'pus_cell', 'coronary_artery_disease', 'white_blood_cell_count', 'class', 'packed_cell_volume'}


In [10]:
for i in catcols:
    print("Columns :",i)
    print(c(data[i])) #using counter for checking the no of classes in the column
    print('*'*120+'\n')

Columns : red_blood_cells
Counter({'normal': 201, nan: 152, 'abnormal': 47})
************************************************************************************************************************

Columns : pedal_edema
Counter({'no': 323, 'yes': 76, nan: 1})
************************************************************************************************************************

Columns : diabetesmellitus
Counter({'no': 258, 'yes': 134, '\tno': 3, '\tyes': 2, nan: 2, ' yes': 1})
************************************************************************************************************************

Columns : appetite
Counter({'good': 317, 'poor': 82, nan: 1})
************************************************************************************************************************

Columns : hypertension
Counter({'no': 251, 'yes': 147, nan: 2})
************************************************************************************************************************

Columns : anemia
Counte

<h1>Removing the columns which are not categorical</h1>

In [11]:
#remove is used for removing the column
catcols.remove('red_blood_cell_count')
catcols.remove('packed_cell_volume')
catcols.remove('white_blood_cell_count')
print(catcols)


{'red_blood_cells', 'pedal_edema', 'diabetesmellitus', 'appetite', 'hypertension', 'anemia', 'bacteria', 'pus_cell_clumps', 'pus_cell', 'coronary_artery_disease', 'class'}


In [12]:
# only fetch the float and int type columns
contcols = set(data.dtypes[data.dtypes!='O'].index.values)
print(contcols)

{'blood glucose random', 'specific_gravity', 'sugar', 'blood_pressure', 'potassium', 'serum_creatinine', 'sodium', 'hemoglobin', 'age', 'albumin', 'blood_urea'}


In [13]:
for i in contcols:
    print("Continous Columns:",i)
    print(c(data[i]))#using counter for checking the number of classes in the column
    print('*'*120+'\n')

Continous Columns: blood glucose random
Counter({99.0: 10, 100.0: 9, 93.0: 9, 107.0: 8, 117.0: 6, 140.0: 6, 92.0: 6, 109.0: 6, 131.0: 6, 130.0: 6, 70.0: 5, 114.0: 5, 95.0: 5, 123.0: 5, 124.0: 5, 102.0: 5, 132.0: 5, 104.0: 5, 125.0: 5, 122.0: 5, 121.0: 4, 106.0: 4, 76.0: 4, 91.0: 4, 129.0: 4, 133.0: 4, 94.0: 4, 88.0: 4, 118.0: 4, 139.0: 4, 111.0: 4, 113.0: 4, 120.0: 4, 119.0: 4, 74.0: 3, 108.0: 3, 171.0: 3, 137.0: 3, 79.0: 3, 150.0: 3, 112.0: 3, 127.0: 3, 219.0: 3, 172.0: 3, 89.0: 3, 128.0: 3, 214.0: 3, 105.0: 3, 78.0: 3, 103.0: 3, 82.0: 3, 97.0: 3, 81.0: 3, 138.0: 2, 490.0: 2, 208.0: 2, 98.0: 2, 204.0: 2, 207.0: 2, 144.0: 2, 253.0: 2, 141.0: 2, 86.0: 2, 360.0: 2, 163.0: 2, 158.0: 2, 165.0: 2, 169.0: 2, 210.0: 2, 101.0: 2, 153.0: 2, 213.0: 2, 424.0: 2, 303.0: 2, 192.0: 2, 80.0: 2, 110.0: 2, 96.0: 2, 85.0: 2, 83.0: 2, 75.0: 2, nan: 1, 423.0: 1, 410.0: 1, 380.0: 1, 157.0: 1, 263.0: 1, 173.0: 1, nan: 1, nan: 1, nan: 1, 156.0: 1, 264.0: 1, nan: 1, 159.0: 1, 270.0: 1, nan: 1, nan: 1, nan: 1,

<h1>Removing the columns which are not numerical</h1>

In [14]:
contcols.remove('specific_gravity')
contcols.remove('albumin')
contcols.remove('sugar')
print(contcols)

{'blood glucose random', 'blood_pressure', 'potassium', 'serum_creatinine', 'sodium', 'hemoglobin', 'age', 'blood_urea'}


<h1>Adding columns which we found continous</h1>

In [15]:
#using add we can add columns
contcols.add('red_blood_cell_count')
contcols.add('packed_cell_volume')
contcols.add('white_blood_cell_count')
print(contcols)

{'blood glucose random', 'blood_pressure', 'potassium', 'serum_creatinine', 'sodium', 'hemoglobin', 'age', 'white_blood_cell_count', 'blood_urea', 'packed_cell_volume', 'red_blood_cell_count'}


<h1>Adding columns which we found categorical</h1>


In [16]:
catcols.add('specific_gravity')
catcols.add('albumin')
catcols.add('sugar')
print(catcols)

{'red_blood_cells', 'pedal_edema', 'diabetesmellitus', 'appetite', 'hypertension', 'anemia', 'bacteria', 'specific_gravity', 'sugar', 'pus_cell_clumps', 'albumin', 'pus_cell', 'coronary_artery_disease', 'class'}


<h1>Rectifying the categorical column classes</h1>

In [17]:
data['coronary_artery_disease']=data.coronary_artery_disease.replace('\tno','no')
c(data['coronary_artery_disease'])

Counter({'no': 364, 'yes': 34, nan: 2})

In [18]:
data['diabetesmellitus']=data.diabetesmellitus.replace(to_replace={'\tno':'no','\tyes':'yes',' yes':'yes'})
c(data['diabetesmellitus'])

Counter({'yes': 137, 'no': 261, nan: 2})

<h1>Null Values</h1>

In [19]:
# it will return if any null vales values present
data.isnull().any()

age                         True
blood_pressure              True
specific_gravity            True
albumin                     True
sugar                       True
red_blood_cells             True
pus_cell                    True
pus_cell_clumps             True
bacteria                    True
blood glucose random        True
blood_urea                  True
serum_creatinine            True
sodium                      True
potassium                   True
hemoglobin                  True
packed_cell_volume          True
white_blood_cell_count      True
red_blood_cell_count        True
hypertension                True
diabetesmellitus            True
coronary_artery_disease     True
appetite                    True
pedal_edema                 True
anemia                      True
class                      False
dtype: bool

In [20]:
#returns the count
data.isnull().count()

age                        400
blood_pressure             400
specific_gravity           400
albumin                    400
sugar                      400
red_blood_cells            400
pus_cell                   400
pus_cell_clumps            400
bacteria                   400
blood glucose random       400
blood_urea                 400
serum_creatinine           400
sodium                     400
potassium                  400
hemoglobin                 400
packed_cell_volume         400
white_blood_cell_count     400
red_blood_cell_count       400
hypertension               400
diabetesmellitus           400
coronary_artery_disease    400
appetite                   400
pedal_edema                400
anemia                     400
class                      400
dtype: int64

In [21]:
data.packed_cell_volume = pd.to_numeric(data.packed_cell_volume , errors='coerce')
data.red_blood_cell_count = pd.to_numeric(data.red_blood_cell_count  , errors='coerce')
data.white_blood_cell_count = pd.to_numeric(data.white_blood_cell_count , errors='coerce')

<h1>Handling Continous/Numerical Columns Missing Values</h1>

In [22]:
data['blood glucose random'].fillna(data['blood glucose random'].mean(),inplace = True)
data['blood_pressure'].fillna(data['blood_pressure'].mean(),inplace = True)
data['blood_urea'].fillna(data['blood_urea'].mean(),inplace = True)
data['hemoglobin'].fillna(data['hemoglobin'].mean(),inplace = True)
data['packed_cell_volume'].fillna(data['packed_cell_volume'].mean(),inplace = True)
data['potassium'].fillna(data['potassium'].mean(),inplace = True)
data['red_blood_cell_count'].fillna(data['red_blood_cell_count'].mean(),inplace = True)
data['serum_creatinine'].fillna(data['serum_creatinine'].mean(),inplace = True)
data['sodium'].fillna(data['sodium'].mean(),inplace = True)
data['white_blood_cell_count'].fillna(data['white_blood_cell_count'].mean(),inplace = True)


In [23]:
data['age'].fillna(data['age'].mode()[0], inplace=True)
data['specific_gravity'].fillna(data['specific_gravity'].mode()[0], inplace=True)
data['albumin'].fillna(data['albumin'].mode()[0], inplace=True)
data['sugar'].fillna(data['sugar'].mode()[0], inplace=True)
data['red_blood_cells'].fillna(data['red_blood_cells'].mode()[0], inplace=True)
data['pus_cell'].fillna(data['pus_cell'].mode()[0], inplace=True)
data['pus_cell_clumps'].fillna(data['pus_cell_clumps'].mode()[0], inplace=True)
data['bacteria'].fillna(data['bacteria'].mode()[0], inplace=True)
data['blood glucose random'].fillna(data['blood glucose random'].mode()[0], inplace=True)
data['hypertension'].fillna(data['hypertension'].mode()[0], inplace=True)
data['diabetesmellitus'].fillna(data['diabetesmellitus'].mode()[0], inplace=True)
data['coronary_artery_disease'].fillna(data['coronary_artery_disease'].mode()[0], inplace=True)
data['appetite'].fillna(data['appetite'].mode()[0], inplace=True)
data['pedal_edema'].fillna(data['pedal_edema'].mode()[0], inplace=True)
data['anemia'].fillna(data['anemia'].mode()[0], inplace=True)
data['class'].fillna(data['class'].mode()[0], inplace=True)

In [24]:
data.isna().sum()

age                        0
blood_pressure             0
specific_gravity           0
albumin                    0
sugar                      0
red_blood_cells            0
pus_cell                   0
pus_cell_clumps            0
bacteria                   0
blood glucose random       0
blood_urea                 0
serum_creatinine           0
sodium                     0
potassium                  0
hemoglobin                 0
packed_cell_volume         0
white_blood_cell_count     0
red_blood_cell_count       0
hypertension               0
diabetesmellitus           0
coronary_artery_disease    0
appetite                   0
pedal_edema                0
anemia                     0
class                      0
dtype: int64

<h1>Label encoding</h1>

In [25]:
#importing label encoding from sklearn
from sklearn.preprocessing import LabelEncoder

for i in catcols:  #looping through all the categorical columns
    print("LABEL ENCODING OF :",i)
    LEi = LabelEncoder() #creating an object of label encoder
    print(c(data[i]))  #getting the classes values before transformation
    data[i] = LEi.fit_transform(data[i]) #transforming our text classes to numerical values
    print(c(data[i]))  #geting class values after transformation
    print('*'*100)


LABEL ENCODING OF : red_blood_cells
Counter({'normal': 353, 'abnormal': 47})
Counter({1: 353, 0: 47})
****************************************************************************************************
LABEL ENCODING OF : pedal_edema
Counter({'no': 324, 'yes': 76})
Counter({0: 324, 1: 76})
****************************************************************************************************
LABEL ENCODING OF : diabetesmellitus
Counter({'no': 263, 'yes': 137})
Counter({0: 263, 1: 137})
****************************************************************************************************
LABEL ENCODING OF : appetite
Counter({'good': 318, 'poor': 82})
Counter({0: 318, 1: 82})
****************************************************************************************************
LABEL ENCODING OF : hypertension
Counter({'no': 253, 'yes': 147})
Counter({0: 253, 1: 147})
****************************************************************************************************
LABEL ENCODING OF : anemia


In [26]:
selcols = ['red_blood_cells','pus_cell','blood glucose random','blood_urea','pedal_edema','anemia','diabetesmellitus',
           'coronary_artery_disease']
x = pd.DataFrame(data, columns = selcols)
y = pd.DataFrame(data, columns = ['class'])
print(x.shape)
print(y.shape)

(400, 8)
(400, 1)


In [27]:
data.isna().sum()

age                        0
blood_pressure             0
specific_gravity           0
albumin                    0
sugar                      0
red_blood_cells            0
pus_cell                   0
pus_cell_clumps            0
bacteria                   0
blood glucose random       0
blood_urea                 0
serum_creatinine           0
sodium                     0
potassium                  0
hemoglobin                 0
packed_cell_volume         0
white_blood_cell_count     0
red_blood_cell_count       0
hypertension               0
diabetesmellitus           0
coronary_artery_disease    0
appetite                   0
pedal_edema                0
anemia                     0
class                      0
dtype: int64

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=2)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(320, 8)
(320, 1)
(80, 8)
(80, 1)


In [29]:
from sklearn.ensemble import RandomForestRegressor

 = RandomForestRegressor()

lgr.fit(x_train,y_train)

/tmp/wsuser/ipykernel_211/2471423432.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg.fit(x_train,y_train)


RandomForestRegressor()

In [30]:
y_pred = lgr.predict(x_test)

In [43]:
y_pred1 =reg.predict([[4,123,0,0,1,1,0,0]])
print(y_pred1)
c(y_pred)

0


Counter({0.0: 32,
         0.01: 8,
         1.0: 9,
         0.3611031746031746: 1,
         0.97: 2,
         0.965: 1,
         0.28: 1,
         0.76: 1,
         0.98: 1,
         0.84: 1,
         0.3754563492063492: 1,
         0.12653571428571428: 1,
         0.02: 2,
         0.9: 1,
         0.95: 1,
         0.1217142857142857: 1,
         0.016031746031746032: 1,
         0.94: 1,
         0.7542857142857143: 1,
         0.05: 1,
         0.01888888888888889: 1,
         0.995: 1,
         0.92: 1,
         0.07666666666666667: 1,
         0.99: 2,
         0.6133333333333334: 1,
         0.08458730158730159: 1,
         0.7: 1,
         0.03: 1,
         0.89: 1,
         0.06160714285714285: 1})

In [32]:
!pip install ibm_watson_machine_learning

In [33]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url":"https://us-south.ml.cloud.ibm.com",
                   "apikey":"YbH1Jt5lA7lHvNhNPEUuKjdais91dxeOYYnHDbLraG2b"
}

client = APIClient(wml_credentials)

In [34]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [35]:
space_uid = guid_from_space_name(client,'models')
print("space UID = "+ space_uid)

space UID = b38739bd-a726-446b-b7c4-ad54856954ca


In [36]:
client.set.default_space(space_uid)

'SUCCESS'

In [37]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [38]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [40]:
model_details = client.repository.store_model(model = lgr,meta_props = {
    client.repository.ModelMetaNames.NAME:"Ckd",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [41]:
model_id

 


'0a9d1b7f-06f9-4031-a952-2a3469cba07a'

In [ ]:
accuracy_score(y_test,y_pred)

<h1>Confusion matrix of our model</h1>

In [44]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
pickle.dump(lgr, open('CKD.pkl','wb'))